In [74]:
#import spark
from pyspark.sql import SparkSession
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 ass2 BNPL group 28")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

In [110]:
import pandas as pd
import numpy as np

In [78]:
raw_data_path = "../data/tables/"

In [126]:
# need to install xlrd run pip install xlrd
poa_2011_2016 = pd.read_excel(raw_data_path+"CG_POA_2011_POA_2016.xls",sheet_name = "Table 3", header = 5)
sa2_2011_2016 = pd.read_excel(raw_data_path+"CG_SA2_2011_SA2_2016.xls",sheet_name = "Table 3", header = 5)
poa_sa2 = pd.read_excel(raw_data_path+"1270055006_CG_POSTCODE_2011_SA2_2011.xls",sheet_name = "Table 3", header = 5)

In [127]:
# change to pyspark dataframe
poa_2011_2016.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
sa2_2011_2016.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
poa_sa2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)

In [128]:
poa_sa2['SA2_MAINCODE_2011'] = poa_sa2['SA2_MAINCODE_2011'].apply(np.int64)
sa2_2011_2016['SA2_MAINCODE_2016'] = sa2_2011_2016['SA2_MAINCODE_2016'].apply(np.int64)

In [129]:
poa_2011_2016=spark.createDataFrame(poa_2011_2016) 
sa2_2011_2016=spark.createDataFrame(sa2_2011_2016) 
poa_sa2=spark.createDataFrame(poa_sa2) 

Drop useless columns

In [131]:
poa_2016_2021 = spark.read.csv('../data/tables/poa_2016_2021.csv',header=True)
sa2_2016_2021 = spark.read.csv('../data/tables/sa2_2016_2021.csv',header=True)

In [132]:
common_cols = ['RATIO','PERCENTAGE']
poa_2011_2016 = poa_2011_2016.drop(*common_cols)
sa2_2011_2016 = sa2_2011_2016.drop(*common_cols)
poa_sa2 = poa_sa2.drop(*common_cols)

In [133]:
common_cols_new = ['RATIO_FROM_TO', 'NDIV_TO_REGION_QLTY_INDICATOR', 'OVERALL_QUALITY_INDICATOR', 'BMOS_NULL_FLAG', 'INDIV_TO_REGION_QLTY_INDICATOR']
poa_2016_2021 = poa_2016_2021.drop(*common_cols_new)
sa2_2016_2021 = sa2_2016_2021.drop(*common_cols_new)


In [134]:
table = poa_2011_2016.join(poa_sa2,poa_2011_2016['POA_CODE_2011'] == poa_sa2['POSTCODE'], how = 'inner')

In [135]:
table = table.join(sa2_2011_2016, table['SA2_MAINCODE_2011'] == sa2_2011_2016['SA2_MAINCODE_2011'], how = 'inner')

In [141]:
poa_2016_2021

POA_CODE_2016,POA_NAME_2016,POA_CODE_2021,POA_NAME_2021
0800,0800,0800,0800
0810,0810,0810,0810
0812,0812,0812,0812
0815,0815,0810,0810
0820,0820,0820,0820
0822,0822,0837,0837
0822,0822,0822,0822
0828,0828,0828,0828
0829,0829,0829,0829
0830,0830,0830,0830


In [142]:
table = table.join(sa2_2016_2021, table['SA2_MAINCODE_2016'] == sa2_2016_2021['SA2_MAINCODE_2016'], how = 'inner')
table = table.join(poa_2016_2021, table['POA_CODE_2016'] == poa_2016_2021['POA_CODE_2016'], how = 'inner')

In [148]:
cols = ['POA_NAME_2011','POA_NAME_2016','POA_NAME_2021','POSTCODE.1','POSTCODE','POA_CODE_2016','']
table = table.drop(*cols)

In [150]:
table_new = table['POA_CODE_2021','SA2_CODE_2021','SA2_NAME_2021']

In [155]:
distinct_df = table_new.distinct()

In [160]:
distinct_df.where(distinct_df['POA_CODE_2021']==3000)

POA_CODE_2021,SA2_CODE_2021,SA2_NAME_2021
3000,206041508,Southbank (West) ...
3000,206041125,South Yarra - West
3000,206041505,Melbourne CBD - West
3000,206041119,East Melbourne
3000,206041503,Melbourne CBD - East
3000,206041509,Southbank - East
3000,206041504,Melbourne CBD - N...
3000,206041510,West Melbourne - ...
3000,206041117,Carlton
3000,206041124,Parkville


In [147]:
# remove some columns


POA_CODE_2011,POA_NAME_2011,POA_CODE_2016,POA_NAME_2016,POSTCODE,POSTCODE.1,SA2_MAINCODE_2011,SA2_NAME_2011,SA2_MAINCODE_2011,SA2_NAME_2011,SA2_MAINCODE_2016,SA2_NAME_2016,SA2_MAINCODE_2016,SA2_NAME_2016,SA2_CODE_2021,SA2_NAME_2021,POA_CODE_2016,POA_NAME_2016,POA_CODE_2021,POA_NAME_2021
2487,2487.0,2487.0,2487.0,2487,2487.0,112031250,Kingscliff - Fing...,112031250,Kingscliff - Fing...,112031250,Kingscliff - Fing...,112031250,Kingscliff - Fing...,112031250,Kingscliff - Fing...,2487,2487,2487,2487
3052,3052.0,3050.0,3050.0,3052,3052.0,206041124,Parkville,206041124,Parkville,206041124,Parkville,206041124,Parkville,206041124,Parkville,3050,3050,3052,3052
3052,3052.0,3052.0,3052.0,3052,3052.0,206041124,Parkville,206041124,Parkville,206041124,Parkville,206041124,Parkville,206041124,Parkville,3052,3052,3052,3052
3000,3000.0,3000.0,3000.0,3000,3000.0,206041124,Parkville,206041124,Parkville,206041124,Parkville,206041124,Parkville,206041124,Parkville,3000,3000,3000,3000
3000,3000.0,3053.0,3053.0,3000,3000.0,206041124,Parkville,206041124,Parkville,206041124,Parkville,206041124,Parkville,206041124,Parkville,3053,3053,3053,3053
3010,3010.0,3010.0,3010.0,3010,3010.0,206041124,Parkville,206041124,Parkville,206041124,Parkville,206041124,Parkville,206041124,Parkville,3010,3010,3052,3052
3758,3758.0,3758.0,3758.0,3758,3758.0,209041224,Wallan,209041224,Wallan,209041224,Wallan,209041224,Wallan,209041224,Wallan,3758,3758,3758,3758
3758,3758.0,3762.0,3762.0,3758,3758.0,209041224,Wallan,209041224,Wallan,209041224,Wallan,209041224,Wallan,209041224,Wallan,3762,3762,3762,3762
3658,3658.0,3658.0,3658.0,3658,3658.0,209041224,Wallan,209041224,Wallan,209041224,Wallan,209041224,Wallan,209041224,Wallan,3658,3658,3658,3658
3658,3658.0,3658.0,3658.0,3658,3658.0,209041224,Wallan,209041224,Wallan,209041224,Wallan,209041224,Wallan,209041224,Wallan,3658,3658,3717,3717
